In [ ]:
import cos.config

import requests
from pytz import UTC
from datetime import datetime, timedelta
from threading import Thread, Timer
import time
import sched
import re
import sounddevice as sd
import scipy

In [ ]:
class Reporter:
    def __init__(self):
        class Words:
            zero = Sample(cos.config.get("audio_files", "zero"))
            one = Sample(cos.config.get("audio_files", "one"))
            two = Sample(cos.config.get("audio_files", "two"))
            three = Sample(cos.config.get("audio_files", "three"))
            four = Sample(cos.config.get("audio_files", "four"))
            five = Sample(cos.config.get("audio_files", "five"))
            six = Sample(cos.config.get("audio_files", "six"))
            seven = Sample(cos.config.get("audio_files", "seven"))
            eight = Sample(cos.config.get("audio_files", "eight"))
            nine = Sample(cos.config.get("audio_files", "nine"))
            minus = Sample(cos.config.get("audio_files", "minus"))
            celsius = Sample(cos.config.get("audio_files", "celsius"))
            current_time_is = Sample(cos.config.get("audio_files", "current_time_is"))
            local_time = Sample(cos.config.get("audio_files", "local_time"))
            minimum_temperature = Sample(
                cos.config.get("audio_files", "minimum_temperature")
            )
            temp_dp_diff = Sample(cos.config.get("audio_files", "temp_dp_diff"))
            you_may_fly = Sample(cos.config.get("audio_files", "you_may_fly"))

        self.words = Words()

    def say_digit(self, digit):
        digit = int(digit)
        [
            self.words.zero,
            self.words.one,
            self.words.two,
            self.words.three,
            self.words.four,
            self.words.five,
            self.words.six,
            self.words.seven,
            self.words.eight,
            self.words.nine,
        ][digit].play()

    def say_number(self, num):
        if num < 0:
            self.words.minus.play()
        for digit in str(abs(num)):
            self.say_digit(digit)

    def say_time(self):
        self.words.current_time_is.play()
        dt = datetime.now().strftime("%H%M")
        for digit in dt:
            self.say_digit(digit)
        self.words.local_time.play()

    def say_min_temperature(self, min_temperature):
        self.words.minimum_temperature.play()
        self.say_number(min_temperature)
        self.words.celsius.play()

    def say_min_separation(self, min_separation):
        self.words.temp_dp_diff.play()
        self.say_number(min_separation)
        self.words.celsius.play()

    def say_you_can_fly(self):
        self.words.you_may_fly.play()


class Sample:
    def __init__(self, filepath):
        sample_frequency, signal = scipy.io.wavfile.read(filepath)
        self.sample_frequency = sample_frequency
        self.signal = signal

    def play(self):
        sd.play(self.signal, self.sample_frequency, blocking=True)


class LangleyReport:
    def __init__(self, metar):

        self.metar = metar

        ac_ident, time_of_record, winds, temp_dp, altimeter = re.search(
            "LWIS (\w+) (\w+)Z AUTO (\w+)KT ([-/\w]+) A(\d\d\d\d)=", metar
        ).groups()
        self.ac_ident = ac_ident

        day = int(time_of_record[0:2])
        hour = int(time_of_record[2:4])
        minute = int(time_of_record[4:])
        now = datetime.now(tz=UTC)
        year, month = now.year, now.month
        self.timestamp = datetime(year, month, day, hour, minute, tzinfo=UTC)

        self.wind_dir = winds[0:3]
        self.wind_speed = winds[3:]

        temperature, dewpoint = temp_dp.split("/")
        self.temperature = int(temperature.replace("M", "-"))
        self.dewpoint = int(dewpoint.replace("M", "-"))

        self.altimeter = altimeter


class METARChecker(Thread):
    CHECK_DELAY = 300

    def __init__(self):
        super().__init__()
        self._do_stop = False

    def run(self):
        while not self._do_stop:
            time.sleep(self.CHECK_DELAY)
            resp = requests.get(
                "https://plan.navcanada.ca/weather/api/alpha/?site=CYNJ&alpha=metar"
            )
            metar = resp.json()["data"][0]["text"]
            report = LangleyReport(metar)

            if any([report.timestamp == r.timestamp for r in weather_reports]):
                continue
            weather_reports.append(report)
            print(f"Capturing {report.metar}")

    def stop(self):
        self._do_stop = True

In [ ]:
weather_reports = []
x = METARChecker()
x.start()

In [ ]:
time_to_notify = datetime(year=2024, month=4, day=1, hour=5, minute=30)
seconds_til_alarm = (time_to_notify - datetime.now()).seconds

In [ ]:
def deliver_report():
    tone_before_report = Sample(cos.config.get("audio_files", "tone_before_report"))
    out_of_bed_music = Sample(cos.config.get("audio_files", "out_of_bed_music"))

    reporter = Reporter()
    temperatures = [r.temperature for r in weather_reports]
    dewpoints = [r.dewpoint for r in weather_reports]
    closeness = [r.temperature - r.dewpoint for r in weather_reports]
    min_temp = min(temperatures)
    min_diff = min(closeness)

    if temperatures[-1] < 0 and min_diff < 1:
        return

    tone_before_report.play()
    reporter.say_time()
    reporter.say_min_temperature(min_temp)
    reporter.say_min_separation(min_diff)
    reporter.say_you_can_fly()

    out_of_bed_music.play()


notifier = Timer(seconds_til_alarm, deliver_report)
notifier.start()

In [ ]:
notifier.cancel()